In [ ]:
import pandas as pd
# import numpy as np
import requests as req
import sys
import time
# pip install holidays if you don't have the package already
import holidays
from datetime import datetime
from datetime import timedelta
# change the following filepath, file, and key name to your own
sys.path.insert(0, '../../API_Keys/')
from vantage_api import alpha_vantage_key

In [19]:
#below, be sure to change the name of 'apikey' to your key's name
def pull_all_stocks(df):
    stock_dict = {}
    for ticker in list(df['ticker'].unique()):
        json = req.get(url = "https://www.alphavantage.co/query",
                  params = {
                      'function': 'TIME_SERIES_DAILY',
                      'outputsize': 'full',
                      'symbol': ticker,
                      'apikey':alpha_vantage_key #change the name of the key to your key name
                  }).json()
        ticker_df = pd.DataFrame(json['Time Series (Daily)']).T
        ticker_df.rename(columns = {'1. open': 'open',
                                 '2. high': 'high',
                                 '3. low': 'low',
                                 '4. close': 'close',
                                 '5. volume': 'volume'}, inplace = True)
        if ticker_df.shape[0] > 2500:
            ticker_df = ticker_df.iloc[range(2500)] #Data beyond 2012 is unnecessary, this helps reduce file size
        stock_dict[f'{ticker}'] = ticker_df
        time.sleep(2) #Might need to be changed for long list of tickers
    return stock_dict


In [20]:
reps = pd.read_csv('data/cleaned_complete_congress_data.csv')

In [21]:
reps.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL


In [70]:
mcconnell = reps[reps['last_name'] == 'McConnell']
mcconnell.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY


In [71]:
mcconnell.shape

(23, 14)

In [72]:
#stocks = pd.DataFrame(columns = ['ticker','daily_time_series'])
#time_series_df.set_index('ticker', inplace = True)

In [73]:
stocks = pull_all_stocks(mcconnell)

In [74]:
stocks

{'WFC':                open     high      low    close    volume
 2022-07-25  43.5300  43.9100  43.2100  43.5000  20284088
 2022-07-22  43.1700  43.5300  42.6900  43.1700  15965371
 2022-07-21  42.4800  43.3700  42.2800  43.2800  17808953
 2022-07-20  42.5400  43.0250  42.3550  42.7200  19111193
 2022-07-19  41.8800  43.0700  41.7450  42.9000  25724214
 ...             ...      ...      ...      ...       ...
 2012-08-22  34.3300  34.4600  34.1100  34.1800  18862500
 2012-08-21  34.2400  34.7600  34.1000  34.3800  25280400
 2012-08-20  34.0300  34.2000  33.9800  34.0700  18147600
 2012-08-17  34.2500  34.3000  33.9600  34.0300  16927000
 2012-08-16  34.1200  34.2000  33.7800  34.1300  19596400
 
 [2500 rows x 5 columns],
 'VMC':                 open      high       low     close  volume
 2022-07-25  156.3200  156.8000  154.8600  156.4000  591024
 2022-07-22  157.1600  157.8200  154.1300  155.6500  480375
 2022-07-21  153.8600  156.6500  153.1450  156.4600  592335
 2022-07-20  151.5200 

In [75]:
stocks['IR'].loc['2022-07-22'].astype(float)

open           45.00
high           45.19
low            44.18
close          44.54
volume    2200156.00
Name: 2022-07-22, dtype: float64

In [76]:
mcconnell[mcconnell['transaction_date'] == '2019-09-01']

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
2388,2019-09-01,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-09-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY


In [77]:
stocks['WFC'].loc['2012-08-21']

open       34.2400
high       34.7600
low        34.1000
close      34.3800
volume    25280400
Name: 2012-08-21, dtype: object

In [78]:
mcconnell.dtypes

transaction_date     datetime64[ns]
ticker                       object
asset_description            object
type                         object
amount                        int64
name                         object
disclosure_date              object
chamber                      object
first_name                   object
last_name                    object
party                        object
birthday                     object
gender                       object
represents                   object
dtype: object

# Code to Pull Stock Prices
[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns. helped to get this written.

## Transaction Date = Day Market Was Closed
Congresspeople sometimes entered transaction dates for which the market was closed. We presumed that means they entered the trade with their broker (or otherwise) on that date, and that it would be executed the next trading day. That's accounted for within a couple of try/except pairs.

[This stackoverflow](https://stackoverflow.com/a/61964191) helped with setting up the try/except. [This medium article](https://medium.com/analytics-vidhya/python-how-to-automate-bank-holidays-767de0d42d44) showed how us the holidays module. [This article](https://www.geeksforgeeks.org/python-holidays-library/) helped understand how to use it. [This](https://pynative.com/python-get-the-day-of-week/) showed how to get day of week to check if saturday or sunday so we can adjust day of transaction accordingly.

## Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.# Market Closed non-Holidays
2018-12-05 the market was closed for national day of mourning to mark [the passing of George Bush](https://heavy.com/news/2018/12/stock-market-closed-national-day-of-mourning/). Need to add it to the list of holidays.

We presume the holiday transaction date means that the orders were executed the next market day.

## Trades When Stocks Weren't Trading AKA Mitch McConnell Sells Something Not On the Exchange?
Some trades are registered as happening when stocks weren't trading. Specifically Ingersoll-Rand (IR) partial_sales by Mitch McConnell 2017-02-02 and 2016-06-09. 

The data on IR doesn't start until May 8, 2017. [This Motley Fool article](https://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspxhttps://www.fool.com/investing/general/2016/01/04/is-ingersoll-rand-a-buy-for-2016.aspx) makes it appear that Ingersoll-Rand was trading under the ticker TT (or perhaps that the part that wasn't spun into IR is trading as TT now).

Gardner Denver went public in May 2017. In May 2020, it combined with Ingersoll-Rand's HVAC business -- together they started trading under IR. See [this article](https://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstanceshttps://seekingalpha.com/article/4350361-new-ingersoll-rand-debuts-under-challenging-circumstances).

This may be a noteworthy trade. The disclosure wasn't made until May 15, 2017 -- about the time of the appearance of IR on the stock exchange, and the time of Gardner Denver going public, but before they merged (at least, that it was announced).

Weird.

He sold it in June of 2019... we should look at this more closely. In the meantime, I'll estimate it as being from the first IR price.

[This site](https://docs.python.org/2/library/datetime.html#datetime.timedeltahttps://docs.python.org/2/library/datetime.html#datetime.timedelta) showed us timedelta



In [79]:
us_holidays = holidays.UnitedStates()
error_dates = []
holiday_dates = []
weird_holidays = [] #holidays for which market wasn't open the next day
saturdays = []
sundays = []
still_unknown = []
mcconnell['trans_trading_day'] = 1 #assume the day is a trading day unless --> see below
mcconnell['trading_day'] = '' #establish the next date of trading, either equal to trans_date or after
for i in mcconnell.index:
    try:
        mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']].drop('volume').astype(float).mean()
        mcconnell.loc[i, 'trading_day'] = mcconnell.loc[i, 'transaction_date']
    except:
        trans_date = mcconnell.loc[i, 'transaction_date']
        trans_date_datetime = pd.to_datetime(mcconnell.loc[i, 'transaction_date'])
        error_dates.append(trans_date)
        mcconnell.loc[i, 'trans_trading_day'] = 0 #to track which transaction dates weren't trading dates
        
        date_plus_1 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 1))[:10]
        date_plus_2 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 2))[:10]
        date_plus_3 = str(pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 3))[:10]

        if trans_date in us_holidays:
            holiday_dates.append(trans_date)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                mcconnell.loc[i, 'trading_day'] = date_plus_1
            except:
                weird_holidays.append(trans_date) #holidays with more than 1 day until the next trading day
        elif trans_date in ['2018-12-05']: #market closed these days for non-regular holidays (2018-12-05 nat'l day of morning for George Bush)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                mcconnell.loc[i, 'trading_day'] = date_plus_1
            except:
                weird_holidays.append(trans_date) #holidays with more than 1 day until the next trading day
        else:
            #check day of week, if saturday add 2 days, if sunday add 1 day
            if datetime.weekday(trans_date_datetime) == 5:
                saturdays.append(trans_date)
                if date_plus_2 not in us_holidays:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_2].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_2
                else:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_3].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_3
            # was here turing transaction to datetime in line before switching strategy
            elif datetime.weekday(trans_date_datetime) == 6:
                sundays.append(trans_date)
                if date_plus_1 not in us_holidays:
                    print(f'Transaction date: {trans_date}')
                    print(f'Date plus 1: {date_plus_1}')
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_1
                else:
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_2].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_2
            else:
                try: 
                    mcconnell.loc[i, 'price'] = stocks[mcconnell.loc[i, 'ticker']].loc[date_plus_1].drop('volume').astype(float).mean()
                    mcconnell.loc[i, 'trading_day'] = date_plus_1
                except:
                    still_unknown.append([mcconnell.loc[i, 'ticker'], trans_date])
                    print(f'{mcconnell.loc[i, "ticker"]} does not appear to trade on {trans_date}, which should be a day the exchange was open')
                    pass
    pass

WFC does not appear to trade on 2022-06-03 00:00:00, which should be a day the exchange was open
WFC does not appear to trade on 2021-12-03 00:00:00, which should be a day the exchange was open
WFC does not appear to trade on 2021-09-03 00:00:00, which should be a day the exchange was open
WFC does not appear to trade on 2018-06-01 00:00:00, which should be a day the exchange was open
IR does not appear to trade on 2017-02-02 00:00:00, which should be a day the exchange was open
IR does not appear to trade on 2016-06-09 00:00:00, which should be a day the exchange was open


In [80]:
saturdays

[Timestamp('2019-06-01 00:00:00')]

In [81]:
sundays

[Timestamp('2019-09-01 00:00:00')]

In [82]:
error_dates

[Timestamp('2022-06-03 00:00:00'),
 Timestamp('2022-03-03 00:00:00'),
 Timestamp('2021-12-03 00:00:00'),
 Timestamp('2021-09-03 00:00:00'),
 Timestamp('2021-06-03 00:00:00'),
 Timestamp('2021-03-03 00:00:00'),
 Timestamp('2020-12-03 00:00:00'),
 Timestamp('2020-09-02 00:00:00'),
 Timestamp('2020-03-04 00:00:00'),
 Timestamp('2019-12-04 00:00:00'),
 Timestamp('2019-09-01 00:00:00'),
 Timestamp('2019-06-01 00:00:00'),
 Timestamp('2019-06-03 00:00:00'),
 Timestamp('2019-06-03 00:00:00'),
 Timestamp('2019-03-05 00:00:00'),
 Timestamp('2018-12-05 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-06-01 00:00:00'),
 Timestamp('2018-03-05 00:00:00'),
 Timestamp('2017-12-05 00:00:00'),
 Timestamp('2017-09-07 00:00:00'),
 Timestamp('2017-02-02 00:00:00'),
 Timestamp('2016-06-09 00:00:00')]

In [83]:
holiday_dates

[]

In [84]:
weird_holidays

[]

In [85]:
error_dates

[Timestamp('2022-06-03 00:00:00'),
 Timestamp('2022-03-03 00:00:00'),
 Timestamp('2021-12-03 00:00:00'),
 Timestamp('2021-09-03 00:00:00'),
 Timestamp('2021-06-03 00:00:00'),
 Timestamp('2021-03-03 00:00:00'),
 Timestamp('2020-12-03 00:00:00'),
 Timestamp('2020-09-02 00:00:00'),
 Timestamp('2020-03-04 00:00:00'),
 Timestamp('2019-12-04 00:00:00'),
 Timestamp('2019-09-01 00:00:00'),
 Timestamp('2019-06-01 00:00:00'),
 Timestamp('2019-06-03 00:00:00'),
 Timestamp('2019-06-03 00:00:00'),
 Timestamp('2019-03-05 00:00:00'),
 Timestamp('2018-12-05 00:00:00'),
 Timestamp('2018-09-06 00:00:00'),
 Timestamp('2018-06-01 00:00:00'),
 Timestamp('2018-03-05 00:00:00'),
 Timestamp('2017-12-05 00:00:00'),
 Timestamp('2017-09-07 00:00:00'),
 Timestamp('2017-02-02 00:00:00'),
 Timestamp('2016-06-09 00:00:00')]

In [86]:
still_unknown

[['WFC', Timestamp('2022-06-03 00:00:00')],
 ['WFC', Timestamp('2021-12-03 00:00:00')],
 ['WFC', Timestamp('2021-09-03 00:00:00')],
 ['WFC', Timestamp('2018-06-01 00:00:00')],
 ['IR', Timestamp('2017-02-02 00:00:00')],
 ['IR', Timestamp('2016-06-09 00:00:00')]]

In [87]:
mcconnell

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,trans_trading_day,trading_day,price
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,,NaN
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2022-03-04,49.265800
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,,NaN
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,,NaN
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2021-06-04,46.907500
709,2021-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-03-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2021-03-04,37.285000
828,2020-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-12-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-12-04,29.590000
966,2020-09-02,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-09-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-09-03,24.745000
1996,2020-03-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-03-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-03-05,39.370000
2196,2019-12-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-12-12,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2019-12-05,53.317500


In [88]:
mcconnell['purchase-sale'] = 0 #to track if this is a set of transactions that we (appear) have start to finish.

[This site](https://www.geeksforgeeks.org/creating-a-list-of-range-of-dates-in-python/) provided guidance in how to create the date list using list comprehension. [This site](https://stackoverflow.com/questions/67102352/convert-a-timedelta-to-int) showed us `.days`.

In [89]:
mcconnell['price_missing'] = 0

In [90]:
reps.dtypes

transaction_date     datetime64[ns]
ticker                       object
asset_description            object
type                         object
amount                        int64
name                         object
disclosure_date              object
chamber                      object
first_name                   object
last_name                    object
party                        object
birthday                     object
gender                       object
represents                   object
dtype: object

In [91]:
reps['transaction_date'] = pd.to_datetime(reps['transaction_date'])
reps.dtypes

transaction_date     datetime64[ns]
ticker                       object
asset_description            object
type                         object
amount                        int64
name                         object
disclosure_date              object
chamber                      object
first_name                   object
last_name                    object
party                        object
birthday                     object
gender                       object
represents                   object
dtype: object

In [92]:
print(f'First transaction date on file: {reps["transaction_date"].min()}')

First transaction date on file: 2012-09-13 00:00:00


In [93]:
start_date = reps["transaction_date"].min()
end_date = pd.to_datetime('2022-07-16') #the data was pulled on 2022-07-15. 2022-07-06 here ensures that date is in the final list
date_list = [start_date + timedelta(days = num) for num in range((end_date-start_date).days)]

In [94]:
date_list[:5]

[Timestamp('2012-09-13 00:00:00'),
 Timestamp('2012-09-14 00:00:00'),
 Timestamp('2012-09-15 00:00:00'),
 Timestamp('2012-09-16 00:00:00'),
 Timestamp('2012-09-17 00:00:00')]

In [95]:
str(date_list[0])[:10]

'2012-09-13'

In [96]:
date_list = [str(date)[:10] for date in date_list]
date_list[:5]

['2012-09-13', '2012-09-14', '2012-09-15', '2012-09-16', '2012-09-17']

In [97]:
date_list[-1]

'2022-07-15'

In [98]:
for i in range(len(date_list)):
    mcconnell[date_list[i]] = 0

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_28136/4248050129.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell[date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_28136/4248050129.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mcconnell[date_list[i]] = 0
/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_28136/4248050129.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perform

In [99]:
mcconnell.columns

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'name', 'disclosure_date', 'chamber', 'first_name', 'last_name',
       ...
       '2022-07-06', '2022-07-07', '2022-07-08', '2022-07-09', '2022-07-10',
       '2022-07-11', '2022-07-12', '2022-07-13', '2022-07-14', '2022-07-15'],
      dtype='object', length=3612)

In [100]:
#maybe explore further -- should we start our count at a later date?
#also for EDA, could be interesting to look at these outlier dates. What was happening?
#for example, 2020-03-18 was right at the start of lockdown. Did it fall on? after?

reps['transaction_date'].value_counts()

2020-03-18    305
2019-06-24    208
2020-04-02    186
2021-02-16    184
2020-04-14    162
             ... 
2017-08-25      1
2018-03-22      1
2021-06-26      1
2015-12-30      1
2018-04-06      1
Name: transaction_date, Length: 1896, dtype: int64

I'm eliminating the NaNs as I don't see any way to fill that data in right now.

In [103]:
mcconnell.dropna(inplace = True)

In [104]:
mcconnell.isnull().sum()[:20]

transaction_date     0
ticker               0
asset_description    0
type                 0
amount               0
name                 0
disclosure_date      0
chamber              0
first_name           0
last_name            0
party                0
birthday             0
gender               0
represents           0
trans_trading_day    0
trading_day          0
price                0
purchase-sale        0
price_missing        0
2012-09-13           0
dtype: int64

In [105]:
mcconnell['type'].value_counts()

purchase     15
sale_full     2
Name: type, dtype: int64

In [ ]:
#after this we need to identify if there are sales that are end dates for any positions.
#we overwrite this.
#we need to look at sale_full even when no purchse, too.

#we can look at closed positions separately. We can still see if there are patterns to the period
#of time after a sale -- does it go down more than the market as a whole -- even when there's no purchase
#in our records

#for loop
#if a purchase
#start from trade_date+1 for each ticker, 
#pull the average price for that day,
#calculate delta from trade_date price
#fill that into the date

#after this, build new DF that has date index, datetime, sum of delta for column one. S&P 500 delta from start date, column 2.


Right now this isn't filling in all the dates with deltas....
I think because this doesn't account for purchases not associated with sales, yet
WRITE THE PART FOR PURCHASES NOT ASSOCIATED WITH SALES.
ALSO ALL DATES AFTER SALE SHOULD BE

In [46]:
mcconnell.loc[0, 'ticker']

'WFC'

In [47]:
stocks['WFC'].index

Index(['2022-07-22', '2022-07-21', '2022-07-20', '2022-07-19', '2022-07-18',
       '2022-07-15', '2022-07-14', '2022-07-13', '2022-07-12', '2022-07-11',
       ...
       '2012-08-28', '2012-08-27', '2012-08-24', '2012-08-23', '2012-08-22',
       '2012-08-21', '2012-08-20', '2012-08-17', '2012-08-16', '2012-08-15'],
      dtype='object', length=2500)

In [48]:
stocks['WFC'].loc['2012-08-20']

open       34.0300
high       34.2000
low        33.9800
close      34.0700
volume    18147600
Name: 2012-08-20, dtype: object

In [59]:
stocks[mcconnell.loc[0, 'ticker']].loc['2012-09-14']

open       35.6000
high       36.6000
low        35.5600
close      36.1300
volume    42005000
Name: 2012-09-14, dtype: object

In [106]:
mcconnell_backup = mcconnell.copy()

In [107]:
#current version 2022-07-25 1600

#pd.options.mode.chained_assignment = None  # default='warn'

#making a dataframe with the full sales in it
sales = mcconnell[(mcconnell['type'] == 'sale_full')]

#drop partial sales
mcconnell = mcconnell[mcconnell['type'] != 'sale_partial']

#looping through every item in the data frame
for i in mcconnell.index:   
    
    #IF A PURCHASE:
    
    if mcconnell.loc[i, 'type'] == 'purchase':      
        #record purchase date
        purchase_date = mcconnell.loc[i, 'transaction_date']
        #record the ticker
        ticker = mcconnell.loc[i, 'ticker']                       
        
        #IS THIS STOCK SOLD LATER?
        for n in sales.index:

            #yes, this stock is sold later
            if (sales.loc[n, 'ticker'] == ticker) and (sales.loc[n, 'transaction_date'] > purchase_date):
                
                #NOTE: THIS CODE DOESN'T YET ACCOUNT FOR A REP BUYING, SELLING, THEN DOING IT AGAIN LATER
                #WITH THE SAME TICKER

                #record the date it's sold
                sale_date = sales.loc(n, 'transaction_date')
                
                #mark these rows as part of a purchase-sale set in the purchase row
                mcconnell.loc[i, 'purchase-sale'] = 1 
                
                #drop the sale row because it's accounted for in the purchase row
                mcconnell.drop(n, inplace = True)
                
                #fill in the changes in price from purchase date to saledate
                for date in pd.date_range(pd.to_datetime(purchase_date), pd.to_datetime(sale_date)):
                    #find the price on the given date
                    price = stocks[ticker].loc[str(date)[:10]].drop('volume').astype(float).mean()
                    
                    #to account for the fact that some dates don't have associated prices:
                    counter = 0
                    try_date = date
                    while price == 0:
                        mcconnell.loc[i, str(try_date)[:10]] = 0 #fill in 0 for the tried date
                        price = stocks[ticker].loc[str(try_date + timedelta(days = 1))[:10]].drop('volume').astype(float).mean() #making price equal the next day's price
                        counter += 1 #advance counter, which we use to prevent an infinite loop
                        try_date = date + timedelta(days = 1) #advance the try_date for the next attempt
                        if counter > 5000:
                            break
                    
                    #create and fill in an entry for the percent change at the given date
                    try:
                        mcconnell.loc[i, str(date)[:10]] = (price-mcconnell.loc[i, 'price'])/mcconnell.loc[i, 'price'] #(new-old)/old
                    except:
                        mcconnell.loc[i, str(date)[:10]] = 0 #this accounts for weekends and holidays
            
            #no, this stock is not sold later
            else:
                
                #looping through all dates from sale to when data was pulled, July 15, 2022
                for date in pd.date_range(pd.to_datetime(purchase_date), pd.to_datetime('2022-07-15')):
                    try:
                        #find the price on the given date
                        price = stocks[ticker].loc[str(date)[:10]].drop('volume').astype(float).mean()
                    except:
                        #to account for the fact that some dates don't have associated prices:
                        counter = 0
                        try_date = date
                        while price == 0:
                            mcconnell.loc[i, str(try_date)[:10]] = 0 #fill in 0 for the tried date
                            price = stocks[ticker].loc[str(try_date + timedelta(days = 1))[:10]].drop('volume').astype(float).mean() #making price equal the next day price
                            counter += 1 #advance counter, which we use to prevent an infinite loop
                            try_date = date + timedelta(days = 1) #advance the try_date for the next attempt
                            if counter > 5000:
                                break
                    
                    #create and fill in an entry for the percent change at the given date
                    try:
                        mcconnell.loc[i, str(date)[:10]] = (price-mcconnell.loc[i, 'price'])/mcconnell.loc[i, 'price'] #(new-old)/old
                    except:
                        mcconnell.loc[i, str(date)[:10]] = 0 #this accounts for weekends and holidays
    
         
    #if there's a full sale but no corresponding purchase we add price on day 1 of dataframe,
    # 2012-09-13 or the first date after that for which a price is available.
    #then calculate deltas through the sale date
    elif mcconnell.loc[i, 'type'] == 'sale_full':  
        
        #we make sure we only deal with the sales unassociated with a purchase here
        if mcconnell.loc[i, 'purchase-sale'] == 0:
            
            #establish the sale date and the price before looping through
            #including in the eventuality that prices aren't available for the stock until
            #after the beginning of the dataset
            
            #ticker
            ticker = mcconnell.loc[i, 'ticker']
            
            #sale date
            sale_date = mcconnell.loc[i, 'transaction_date'] # date of sale
            
            #establishing sale price as that day or the first day after for which a price is available
            try:
                sale_price = stocks[ticker].loc[sale_date].drop('volume').astype(float).mean()
            except:
                counter = 0
                try_date = sale_date
                while sale_price == 0:
                    mcconnell.loc[i, str(try_date)[:10]] = 0 #fill in 0 for the tried date
                    sale_price = stocks[ticker].loc[str(try_date + timedelta(days = 1))[:10]].drop('volume').astype(float).mean() #making price equal the next day price
                    counter += 1 #advance counter, which we use to prevent an infinite loop
                    try_date = date + timedelta(days = 1) #advance the try_date for the next attempt
                    if counter > 5000:
                        break
            
            #establishing the starting price, either 2012-09-13 or the first date after
            #that a price is available for that stock.
            #establish start_date outside loop, else statement will modify it
            start_date = date
            try:
                start_price = stocks[mcconnell.loc[i, 'ticker']].loc['2012-09-13'].drop('volume').astype(float).mean() #by default, replaced if first_date is later
            except:
                counter = 0
                start_date = date
                while start_price == 0:
                    mcconnell.loc[i, str(start_date)[:10]] = 0 #fill in 0 for the tried date
                    start_price = stocks[ticker].loc[str(try_date + timedelta(days = 1))[:10]].drop('volume').astype(float).mean() #making price equal the next day price
                    counter += 1 #advance counter, which we use to prevent an infinite loop
                    start_date = date + timedelta(days = 1) #advance the try_date for the next attempt
                    if counter > 5000:
                        break
                mcconnell.loc[i, 'price'] = start_price
            
            #looping through the relevant dates, from start to actual excecution ('try date')
            #filling in the percent change
            for date in pd.date_range(pd.to_datetime(start_date), pd.to_datetime(try_date)):
                
                #this shouldn't need a try date because it should already have been 
                price = stocks[ticker].loc[date].drop('volume').astype(float).mean()
                try:
                    mcconnell.loc[i, str(date)[:10]] = (price-mcconnell.loc[i, 'price'])/mcconnell.loc[i, 'price'] #(new-old)/old
                except:
                    mcconnell.loc[i, str(date)[:10]] = 0 #this accounts for weekends and holidays
            
            

In [ ]:
mcconnell['price'].dtypes

In [ ]:
(datetime('2012-07-12')+timedelta(days=1))

In [ ]:
pd.set_option('display.max_columns', None)

In [108]:
mcconnell

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,trans_trading_day,trading_day,price,purchase-sale,price_missing,2012-09-13,2012-09-14,2012-09-15,2012-09-16,2012-09-17,2012-09-18,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05,2022-07-06,2022-07-07,2022-07-08,2022-07-09,2022-07-10,2022-07-11,2022-07-12,2022-07-13,2022-07-14,2022-07-15
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2022-03-04,49.265800,0,0,0,0,0,0,0,0,...,-0.199977,-0.216647,-0.231363,-0.192899,-0.192899,-0.192899,-0.178106,-0.173920,-0.187824,-0.207209,-0.202388,-0.202388,-0.202388,-0.202388,-0.199597,-0.197897,-0.185312,-0.182826,-0.182826,-0.182826,-0.194649,-0.197415,-0.208833,-0.223549,-0.180111
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2021-06-04,46.907500,0,0,0,0,0,0,0,0,...,-0.159756,-0.177264,-0.192720,-0.152321,-0.152321,-0.152321,-0.136785,-0.132388,-0.146991,-0.167351,-0.162287,-0.162287,-0.162287,-0.162287,-0.159356,-0.157571,-0.144353,-0.141742,-0.141742,-0.141742,-0.154160,-0.157064,-0.169057,-0.184512,-0.138890
709,2021-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-03-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2021-03-04,37.285000,0,0,0,0,0,0,0,0,...,0.057094,0.035068,0.015623,0.066448,0.066448,0.066448,0.085993,0.091525,0.073153,0.047539,0.053909,0.053909,0.053909,0.053909,0.057597,0.059842,0.076472,0.079757,0.079757,0.079757,0.064134,0.060480,0.045393,0.025949,0.083345
828,2020-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-12-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-12-04,29.590000,0,0,0,0,0,0,0,0,...,0.331996,0.304241,0.279740,0.343782,0.343782,0.343782,0.368410,0.375380,0.352230,0.319956,0.327982,0.327982,0.327982,0.327982,0.332629,0.335459,0.356413,0.360553,0.360553,0.360553,0.340867,0.336262,0.317252,0.292751,0.365073
966,2020-09-02,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-09-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-09-03,24.745000,0,0,0,0,0,0,0,0,...,0.592797,0.559608,0.530309,0.606890,0.606890,0.606890,0.636341,0.644676,0.616993,0.578400,0.587998,0.587998,0.587998,0.587998,0.593554,0.596938,0.621994,0.626945,0.626945,0.626945,0.603405,0.597899,0.575166,0.545868,0.632350
1996,2020-03-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2020-03-11,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2020-03-05,39.370000,0,0,0,0,0,0,0,0,...,0.001111,-0.019749,-0.038164,0.009970,0.009970,0.009970,0.028480,0.033719,0.016320,-0.007938,-0.001905,-0.001905,-0.001905,-0.001905,0.001588,0.003714,0.019463,0.022574,0.022574,0.022574,0.007779,0.004318,-0.009970,-0.028385,0.025972
2196,2019-12-04,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-12-12,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2019-12-05,53.317500,0,0,0,0,0,0,0,0,...,-0.260773,-0.276176,-0.289774,-0.254232,-0.254232,-0.254232,-0.240564,-0.236695,-0.249543,-0.267454,-0.263000,-0.263000,-0.263000,-0.263000,-0.260421,-0.258851,-0.247222,-0.244924,-0.244924,-0.244924,-0.255849,-0.258405,-0.268955,-0.282553,-0.242416
2388,2019-09-01,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-09-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,0,2019-09-03,46.055000,0,0,0,0,0,0,0,0,...,-0.144203,-0.162035,-0.177777,-0.136630,-0.136630,-0.136630,-0.120807,-0.116328,-0.131202,-0.151938,-0.146781,-0.146781,-0.146781,-0.146781,-0.143795,-0.141978,-0.128515,-0.125855,-0.125855,-0.125855,-0.138503,-0.141461,-0.153675,-0.169417,-0.122951
2541,2019-06-01,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,

In [51]:
pd.set_option('display.max_columns', 3500)

In [52]:
pd.set_option('display.max_columns', 50)

In [53]:
#need to account for once sale happens the value maintaining through end as far as delta goes.
mcconnell[mcconnell['type'] == 'sale_full']

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,trans_trading_day,trading_day,price,purchase-sale,price_missing
2542,2019-06-03,VMC,Vulcan Materials Company,sale_full,500000,A. Mitchell Mcconnell,2019-06-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,1,2019-06-03,126.805,0,0
2543,2019-06-03,IR,Ingersoll-Rand Plc,sale_full,500000,A. Mitchell Mcconnell,2019-06-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,1,2019-06-03,34.040,0,0


In [54]:
mcconnell[mcconnell['type'] == 'sale_full']['price_missing']

2542    0
2543    0
Name: price_missing, dtype: int64

[This site](https://www.geeksforgeeks.org/python-iterating-through-a-range-of-dates/) showed us the date_range method.

In [55]:
 #a draft from above maybe useful elsewhere?
    #HOWEVER, IN ANOTHER ANALYSIS WE MAY WANT TO KEEP THESE TO LOOK AT PATTERNS OF
                                        #PRICES DROPPING OR NOT AFTER SALES
  #              for date in pd.date_range(sale_date + timedelta(days = 1, datetime('2022-07-15'))):
   #                 mcconnell.loc[i, [str(date)[:10]] = 0

In [56]:
mcconnell.columns[:20]

Index(['transaction_date', 'ticker', 'asset_description', 'type', 'amount',
       'name', 'disclosure_date', 'chamber', 'first_name', 'last_name',
       'party', 'birthday', 'gender', 'represents', 'trans_trading_day',
       'trading_day', 'price', 'purchase-sale', 'price_missing'],
      dtype='object')

In [192]:
stocks['IR']#.loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

,open,high,low,close,volume
2022-07-22,45.0000,45.1900,44.1800,44.5400,2200156
2022-07-21,44.0700,44.9900,43.7300,44.9600,1633376
2022-07-20,43.2900,44.2400,43.1400,44.0700,2715744
2022-07-19,41.8300,43.8100,41.8300,43.2700,4499483
2022-07-18,41.5000,42.1600,40.9200,41.2200,3052687
...,...,...,...,...,...
2017-05-18,22.1900,23.6300,21.9800,22.5000,4075173
2017-05-17,22.1500,22.5100,21.6700,22.1000,1583699
2017-05-16,22.1900,22.4100,22.0500,22.1000,1289339
2017-05-15,21.0100,22.3000,21.0100,22.0200,3487278


In [180]:
stocks[mcconnell.loc[0, 'ticker']].loc['2017-02-02']#.loc[mcconnell.loc[0, 'transaction_date']]

open       55.5650
high       55.9600
low        55.4000
close      55.7500
volume    13044449
Name: 2017-02-02, dtype: object

In [ ]:
stocks[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, date_plus_2]]

In [174]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float)

open     44.8300
high     45.3299
low      44.6100
close    44.8200
Name: 2022-06-03, dtype: float64

This pulls the average of open, high, low, and close. Using this because we have no idea when in the day the purchase was made.

In [176]:
stocks[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float).mean()

44.897475